# Excel Parser

Class TabularDataExtractor
- Excel File
- CSV
- PDF

Each of these should extract data into a pandas dataframe, which can then be cleaned.

Document
:param dataframe: Pandas dataframe with the document's content, if the document contains tabular data.
:param meta: Additional custom metadata for the document. Must be JSON-serializable.
    - number of Nas per row, length of row


- Read Excel File
- Put each sheet into a list

In [3]:
test_filepath = "raw_files/CMS SEAS - Clean PB_KH.xlsx"

In [4]:
import pandas as pd
import os

def find_tables(sheet_df):
    """ 
    Function gets the length of the first row with non-null values and uses that as the length of the table.
    It then iterates through the rows and appends them to the table if they have the same length as the first row.
    """
    tables = []
    table = None
    current_row_length = 0
    
    for index, row in sheet_df.iterrows():
        non_null_count = row.count()
        
        if non_null_count == 0:  # Skip rows with all nulls
            continue
        
        if table is None:
            current_row_length = non_null_count
            table = {'header': row.index[row.notna()].tolist(), 'data': [row.dropna().tolist()]}
        elif non_null_count == current_row_length:
            table['data'].append(row.dropna().tolist())
        else:
            if table is not None:
                tables.append(table)
                table = None
                
            if non_null_count > 0:
                current_row_length = non_null_count
                table = {'header': row.index[row.notna()].tolist(), 'data': [row.dropna().tolist()]}

    if table is not None:
        tables.append(table)
        
    return tables

def save_tables_to_csv(tables, sheet_name, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    for i, table in enumerate(tables):
        table_df = pd.DataFrame(table['data'], columns=table['header'])
        table_df.to_csv(f"{output_folder}/table{i+1}.csv", index=False)

def parse_excel_file(file_path):
    excel_data = pd.ExcelFile(file_path)
    
    for sheet_name in excel_data.sheet_names:
        sheet_df = pd.read_excel(file_path, sheet_name=sheet_name)
        tables = find_tables(sheet_df)
        save_tables_to_csv(tables, sheet_name, output_folder=sheet_name)

In [ ]:
# Example usage
parse_excel_file(test_filepath)

In [2]:
!ls Sprint\ Team\ -\ Base

table1.csv  table14.csv table19.csv table23.csv table28.csv table6.csv
table10.csv table15.csv table2.csv  table24.csv table29.csv table7.csv
table11.csv table16.csv table20.csv table25.csv table3.csv  table8.csv
table12.csv table17.csv table21.csv table26.csv table4.csv  table9.csv
table13.csv table18.csv table22.csv table27.csv table5.csv
